# [M_01] PROTOKÓŁ: ANALIZA MULTIMODALNA (GEMINI 3 FLASH)

**PROJEKT:** OMNI-OPERATOR-V1  
**ORGANIZACJA:** [KUŹNIA OPERATORÓW](https://takzenai-hub.pl)  
**STATUS:** OPERACJA_WIDZENIA

Ten moduł implementuje logikę multimodalnej analizy surowego materiału wideo. Wykorzystujemy model **Gemini 3 Flash**, aby bezpośrednio wyekstrahować kluczowe momenty (hooki) oraz strukturę narracyjną. 

**Dlaczego to jest przełomowe?**
1. **Brak transkrypcji:** Nie tracimy czasu na Whisper/STT. Gemini widzi gesty, emocje i tekst na ekranie.
2. **Video Grounding:** Model łączy dźwięk z konkretnymi klatkami obrazu.
3. **Structured Outputs:** Wynik trafia bezpośrednio do modelu Pydantic, gotowy do automatycznego montażu.

In [1]:
import os
import sys
import time
import json
from google import genai  
from pydantic import BaseModel, Field
from typing import List

# 1. KOREKTA ŚCIEŻKI ROBOCZEJ
if os.getcwd().endswith("notebooks"):
    os.chdir("..")

# Dodanie src do path, aby widzieć moduł core
sys.path.append(os.path.join(os.getcwd(), "src"))

from src.core.config import settings

# 2. KONFIGURACJA SILNIKA
client = genai.Client(api_key=settings.gemini_api_key)  

print(f"LOG: System analityczny gotowy. Katalog ROOT: {os.getcwd()}")

LOG: System analityczny gotowy. Katalog ROOT: c:\Users\takze\OneDrive\Pulpit\project\omni-operator-v1


## 1. Definicja Kontraktu Danych (Structured Output)

Ustalenie ścisłego schematu jest kluczowe dla Etapu 3 (Montaż FFmpeg). Gemini musi zwrócić dane w formacie, który Python zrozumie bezbłędnie.

In [2]:
class ShotCandidate(BaseModel):
    """Reprezentuje fragment wideo wyselekcjonowany pod kątem viralowym."""
    start: str = Field(description="Znacznik czasu rozpoczęcia fragmentu (format MM:SS)")
    end: str = Field(description="Znacznik czasu zakończenia fragmentu (format MM:SS)")
    visual_description: str = Field(description="Opis tego, co dzieje się na obrazie w tym momencie")
    narrative_hook: str = Field(description="Dlaczego ten moment przyciągnie uwagę widza")
    score: int = Field(description="Potencjał viralowy w skali 1-10")

class VideoAnalysisReport(BaseModel):
    """Kompletny raport z analizy materiału źródłowego."""
    main_topic: str = Field(description="Główny temat i cel nagrania")
    suggested_titles: List[str] = Field(description="Propozycje chwytliwych tytułów (max 3)")
    clips: List[ShotCandidate] = Field(description="Lista sugerowanych fragmentów do wycięcia")

print("LOG: Modele danych Pydantic zainicjalizowane.")

LOG: Modele danych Pydantic zainicjalizowane.


## 2. Zarządzanie Plikami Mediów (Google File API)

Przed analizą plik wideo musi zostać zaindeksowany przez Google. Gemini 3 Flash wymaga, aby plik posiadał status `ACTIVE`.

In [3]:
def upload_to_gemini(file_path: str):
    """Przesyła plik do API i monitoruje status przetwarzania."""
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Błąd: Nie znaleziono pliku {file_path} w katalogu ROOT.")

    print(f"LOG: Przesyłanie materiału {file_path} do Google Cloud...")
    media_file = genai.upload_file(path=file_path)
    
    # Czekanie na przetworzenie przez serwery Google
    while media_file.state.name == "PROCESSING":
        print(".", end="", flush=True)
        time.sleep(3)
        media_file = genai.get_file(media_file.name)
        
    if media_file.state.name == "FAILED":
        raise RuntimeError("LOG: Przetwarzanie wideo przez Google API zakończone niepowodzeniem.")
        
    print(f"\nLOG: Materiał aktywny. URI: {media_file.uri}")
    return media_file

## 3. Wykonanie Analizy (Native Vision & Reasoning)

Uruchamiamy proces analizy. Używamy mechanizmu `response_schema`, aby wymusić na modelu Gemini 2.5 Flash ścisłe trzymanie się naszej struktury danych.

In [4]:
# KRYTYCZNE: Upewnij się, że plik test_video.mp4 jest w Twoim folderze głównym!
INPUT_FILE = "test_video.mp4"

async def run_multimodal_analysis():
    try:
        # 1. Upload mediów
        video_handle = upload_to_gemini(INPUT_FILE)
        
        # 2. Inicjalizacja modelu
        model = genai.GenerativeModel(model_name="gemini-3-flash-preview") # Zgodnie z API, 2.5 jest dostępne jako 2.5-flash-latest lub exp
        
        prompt = (
            "Przeanalizuj to nagranie wideo pod kątem tworzenia krótkich treści (Shorts/TikTok). "
            "Zidentyfikuj główny temat i wybierz 3 momenty o najwyższym potencjale viralowym. "
            "Zwróć wynik jako czysty JSON zgodny ze strukturą VideoAnalysisReport."
        )
        
        # 3. Analiza z wymuszeniem schematu
        print("LOG: Agent analizuje obraz i dźwięk...")
        response = model.generate_content(
            [prompt, video_handle],
            generation_config={
                "response_mime_type": "application/json",
                "response_schema": VideoAnalysisReport
            }
        )
        
        # 4. Parsowanie i walidacja
        report_data = json.loads(response.text)
        report = VideoAnalysisReport.model_validate(report_data)
        
        # 5. Prezentacja wyników
        print("\n" + "="*45)
        print(f"🚀 RAPORT: {report.main_topic.upper()}")
        print("="*45)
        for i, clip in enumerate(report.clips, 1):
            print(f"CLIP {i}: [{clip.start} - {clip.end}] (Score: {clip.score}/10)")
            print(f"VISUAL: {clip.visual_description}")
            print(f"HOOK: {clip.narrative_hook}\n")
            
        return report

    except Exception as e:
        print(f"❌ BŁĄD OPERACYJNY: {str(e)}")

# URUCHOMIENIE
analysis_result = await run_multimodal_analysis()

LOG: Przesyłanie materiału test_video.mp4 do Google Cloud...
❌ BŁĄD OPERACYJNY: module 'google.genai' has no attribute 'upload_file'


## STATUS: MODUŁ 01 ZAKOŃCZONY

Mamy gotowe dane wejściowe do montażu. System "zrozumiał" wideo i wytypował momenty do wycięcia.

**Kolejne kroki:**
1. Zapisz notatnik.
2. Przejdź do `notebooks/02_AGENT_COPYWRITER.ipynb`.